# LangChain: Q&A over Documents

An example might be a tool that would allow you to query a product catalog for items of interest.

!pip install --upgrade langchain

https://learn.deeplearning.ai/langchain/lesson/5/question-and-answer


In [1]:
import os
from dotenv import load_dotenv

def set_env_path(env_path):
    # Set the path to the .env file
    load_dotenv(dotenv_path=env_path)

# Define the path to the .env file
env_path = "YOUR_ENV_PATH"
# Set the environment variables from the .env file
set_env_path(env_path)

In [2]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

In [7]:
file = '/Users/m.maguga-darbinian/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [8]:
from langchain.indexes import VectorstoreIndexCreator

In [5]:
!pip install docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 4.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.5/240.5 kB 5.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 7.0 MB/s eta 0:00:00


In [9]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch
).from_loaders([loader])

In [11]:
query ="Please list all your items with rain protection \
in a table in markdown and summarize each one."

In [13]:
response = index.query(query)
display(Markdown(response))



| Item | Description | Price | Color | Size |
| --- | --- | --- | --- | --- |
| Rain Poncho | Stay dry during unexpected showers with this lightweight rain poncho. | 24.99 | Yellow | One Size |
| Waterproof Jacket | Stay dry in rainy weather with this waterproof jacket. | 79.99 | Blue | M |
| Outdoor Hat | Protect yourself from the sun with this wide-brimmed outdoor hat. | 19.99 | Khaki | One Size |
| Camping Tent | Spacious camping tent for comfortable outdoor accommodation. | 129.99 | Green | 4 Person |

The items with rain protection are the Rain Poncho, Waterproof Jacket, and Outdoor Hat. The Rain Poncho is a lightweight poncho that will keep you dry during unexpected showers. The Waterproof Jacket is designed to keep you dry in rainy weather. The Outdoor Hat is a wide-brimmed hat that will protect you from the sun.

In [14]:
loader = CSVLoader(file_path=file)

In [15]:
docs = loader.load()

In [16]:
docs[0]

Document(page_content='Product Name: Hiking Pants\nDescription: Durable and lightweight hiking pants for outdoor adventures.\nPrice: 49.99\nColor: Black\nSize: L', metadata={'source': '/Users/m.maguga-darbinian/OutdoorClothingCatalog_1000.csv', 'row': 0})

# Crete embeddings

In [17]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [21]:
embed = embeddings.embed_query("Hi my name is Manuel")
print(len(embed))

1536


In [20]:
print(embed[:5])

[-0.021913960576057434, 0.006774206645786762, -0.018190348520874977, -0.039148248732089996, -0.014089343138039112]


In [22]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [23]:
query = "Please suggest an item with rain protection"

In [24]:
docs = db.similarity_search(query)

In [25]:
len(docs)

4

In [26]:
docs[0]

Document(page_content='Product Name: Waterproof Jacket\nDescription: Stay dry in rainy weather with this waterproof jacket.\nPrice: 79.99\nColor: Blue\nSize: M', metadata={'source': '/Users/m.maguga-darbinian/OutdoorClothingCatalog_1000.csv', 'row': 1})

In [27]:
retriever = db.as_retriever()

In [28]:
llm = ChatOpenAI(temperature = 0.0)

In [29]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [32]:
response = llm.call_as_llm(f"{qdocs} Question: Please list all items \
                            with rain protection in a table in markdown and summarize each one.") 

display(Markdown(response))

| Product Name   | Description                                                  | Price | Color   | Size     |
|----------------|--------------------------------------------------------------|-------|---------|----------|
| Waterproof Jacket | Stay dry in rainy weather with this waterproof jacket. | 79.99 | Blue    | M        |
| Rain Poncho    | Stay dry during unexpected showers with this lightweight rain poncho. | 24.99 | Yellow  | One Size |
| Outdoor Hat    | Protect yourself from the sun with this wide-brimmed outdoor hat. | 19.99 | Khaki   | One Size |
| Hiking Pants   | Durable and lightweight hiking pants for outdoor adventures. | 49.99 | Black   | L        |

- Waterproof Jacket: This jacket is designed to keep you dry in rainy weather. It is made of waterproof material and is suitable for outdoor activities. It is priced at $79.99 and is available in blue color and size M.
- Rain Poncho: This lightweight rain poncho is perfect for unexpected showers. It is easy to carry and provides protection from rain. It is priced at $24.99 and is available in yellow color and one size fits all.
- Outdoor Hat: This wide-brimmed hat is ideal for protecting yourself from the sun. It is suitable for outdoor activities and provides shade and sun protection. It is priced at $19.99 and is available in khaki color and one size fits all.
- Hiking Pants: These pants are designed for outdoor adventures. They are durable and lightweight, making them perfect for hiking. They do not provide rain protection specifically, but they are suitable for outdoor activities. They are priced at $49.99 and are available in black color and size L.

In [33]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [36]:
query =  "Question: Please list all items \
with rain protection in a table in markdown and summarize each one."

In [38]:
response = qa_stuff.run(query)
display(Markdown(response))



> Entering new  chain...

> Finished chain.


| Product Name    | Description                                                  | Price | Color  | Size     |
|-----------------|--------------------------------------------------------------|-------|--------|----------|
| Waterproof Jacket | Stay dry in rainy weather with this waterproof jacket.       | 79.99 | Blue   | M        |
| Rain Poncho     | Stay dry during unexpected showers with this lightweight rain poncho. | 24.99 | Yellow | One Size |
| Outdoor Hat     | Protect yourself from the sun with this wide-brimmed outdoor hat. | 19.99 | Khaki  | One Size |
| Hiking Pants    | Durable and lightweight hiking pants for outdoor adventures. | 49.99 | Black  | L        |

- Waterproof Jacket: A waterproof jacket designed to keep you dry in rainy weather. It is priced at $79.99 and is available in the color blue. The size available is M.
- Rain Poncho: A lightweight rain poncho that provides protection during unexpected showers. It is priced at $24.99 and is available in the color yellow. The size available is one size fits all.
- Outdoor Hat: A wide-brimmed outdoor hat that offers sun protection. It is priced at $19.99 and is available in the color khaki. The size available is one size fits all.
- Hiking Pants: Durable and lightweight hiking pants suitable for outdoor adventures. They are priced at $49.99 and are available in the color black. The size available is L.

In [39]:
response = index.query(query, llm=llm)

In [40]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])